<a href="https://colab.research.google.com/github/LMJ1977/covid/blob/master/extractor_aulas_cerradas_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extracción pedrestre de datos de contagios en colegios, basada en https://muddoo.com/tutorials/how-to-extract-data-from-a-website-using-python/. 


##Carga de librerías utilizadas

In [231]:

import pandas as pd
import re
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
!pip install feedparser
import feedparser

In [232]:
def evaluate_no_number(txt):
  if len(txt)>0:
    number = '1'
  else:
    number = '1' #work in progress
  return number

In [233]:
def search_in_centro(position):
    center_type = []
    for kwd in cen_Keywords:
      if kwd.lower() in data['Centro'][position].lower():
        if kwd == 'CRA':
          kwd = 'Primaria'
        center_type.append(kwd)
      else:
        continue
      return center_type

 web datos colegios http://aragonhoy.net/index.php/mod.noticias/mem.detalle/area.1056/id.265437

##Suscripción a RSS Feed de Aragon Hoy. Filtrado por palabras clave

In [234]:
Feed = feedparser.parse('http://www.aragonhoy.net/index.php/mod.noticias/mem.rss')
#print(len(Feed.entries))
for i in range(len(Feed.entries)):
  pointer = Feed.entries[i]
  if (all(i in pointer.summary.lower() for i in ['coronavirus','aula','cerrada'])):
    print (pointer.summary)
    print (pointer.link)

Cerradas 20 aulas en quince centros educativos de Aragón por casos de coronavirus
http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/id.265807


In [235]:
target_web = pointer.link
#target_web = 'http://aragonhoy.net/index.php/mod.noticias/mem.detalle/area.1056/id.265437'
#target_web = 'http://aragonhoy.net/index.php/mod.noticias/mem.detalle/area.1050/id.265807'
#target_web = 'http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.91/id.265528'
#target_web = 'http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.91/id.265647'
#target_web = 'http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.91/id.265807'
#target_web = 'http://aragonhoy.net/index.php/mod.noticias/mem.detalle/area.1019/id.265713'
#target_web = 'http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.91/id.265289' #Boletin inicial, 200911 no sigue el patrón de los otros boletines. No funciona
content = urlopen(target_web)
read_content = content.read()

##Búsqueda texto en cuerpo de la web y fecha de publicación de la noticia

In [236]:
# parse data with web scrapper
datascrapped = BeautifulSoup(read_content,'html.parser')
soup = datascrapped.find_all('p')              
pubdate = datascrapped.find('span', {'class': 'fec'})
#print('Datos correspondientes al',pubdate.text)

##Definición de cadenas de texto a buscar


In [237]:
cen_Keywords = ['Colegio', 'colegio', 'CEIP', 'CEP', 'CEE','Instituto','CRA', 'Colegio Rural Agrupado','IES', 'Instituto','CPI']
lvl_Keywords = ['Infantil','Primaria','Secundaria', 'Bachillerato','CEE', 'FP']

##Construcción de la tabla de datos

In [238]:
raw = []
for line in soup:
  if (any(cen in line.text for cen in cen_Keywords)):
    raw.append(line.text)

In [239]:
data = pd.DataFrame(raw)

In [240]:
new_data = data[0].str.split(':',n=1, expand = True)
# making separate first name column from new data frame 
data["Centro"]= new_data[0] 
  
# making separate last name column from new data frame 
data["Incidencia"]= new_data[1] 
  
# Dropping old Name columns and rows with no "incidencia" 
data.drop(columns =[0], inplace = True)
data=data[data['Incidencia'].astype(bool)] 
data

,Centro,Incidencia
0,CEIP Gil Tarín (La Muela),1 aula de Infantil
1,CEIP Ciudad de Zaragoza (Zaragoza),1 aula de Infantil
2,CEIP Juan XXIII (Zaragoza),1 aula de Primaria
3,CEIP Lucien Briet (Zaragoza),1 aula de Primaria
4,Colegio Santo Domingo de Silos (Zaragoza),2 aulas de Infantil
5,Colegio María Auxiliadora-Salesianas (Zaragoza),1 aula de Infantil
6,Colegio El Salvador (Zaragoza),2 aulas de Primaria
7,Colegio Escolapias Calasanz (Zaragoza),1 aula de Primaria
8,CRA Los Bañales (Sádaba),1 aula de Primaria
9,CEIP Mateo Valero (Alfamén),1 aula de Primaria


##Cambiamos textos por números, con un poco de picardía...



In [241]:
dictionary = {" UNA ":" 1 ", " OTRA ":" 1 "," DOS ":" 2 "," UNO ":" 1 "," OTRO ":" 1 "," UN PAR ":" 2 "," TRES ":" 3 "," CUATRO ":" 4 ","CINCO ":" 5 ", "ESO":"Secundaria", "BAC":"Bachillerato" } 

for ind,item in enumerate(data['Incidencia']):
  #item = re.sub(r'(?<=[.,])(?=[^\s])', r' ', item)
  #remove parentheses
  item = re.sub('\(', ' ', item)
  item = re.sub('\)',' ',item)
  
  #item = re.sub('\.', ' \. ', item)
  for key in dictionary.keys():
    redata = re.compile(re.escape(key), re.IGNORECASE)
    item = redata.sub(dictionary[key],item)
  #print(item)   
  data['Incidencia'][ind]=item 

##Añadimos a la tabla columnas por niveles educativos

In [242]:
for lvl in lvl_Keywords:
  data[lvl]=''
#data

##Extraemos los números de cada línea y los convertimos de string a entero. Volvemos a recorrer los elementos. Vamos a buscar el nivel al que corresponden buscando el nivel y asignandole la posición a la que corresponde. Si el numero de niveles y el número de cifras obtenidas de la lectura de la línea no coincide, el primer número es el total y el resto el desglose por niveles.

In [243]:
for ind,item in enumerate(data['Incidencia']):
  numbers = re.findall('[0-9]+',item)
  if numbers == []:
    numbers = evaluate_no_number(item)
  intnumbers = [int(i) for i in numbers]
  if len(intnumbers)>1:
    lvl = []
    for kwd in lvl_Keywords:
      pat = re.compile(re.escape(kwd), re.IGNORECASE)
      try:
        pos = re.search(pat,item).start()
        lvl.append(kwd)
      except:
        continue
    if len(lvl)==0:
      lvl = search_in_centro(ind) 
    if len(lvl)==len(intnumbers):
      shift = 0
    else:
      shift = 1
    for idx,l in enumerate(lvl):
        data[l][ind]=intnumbers[idx+shift]  
  else:
    lvl = []
    for kwd in lvl_Keywords:
      pat2 = re.compile(re.escape(kwd), re.IGNORECASE)
      try:
        pos = re.search(pat2,item).start()
        lvl.append(kwd)
        data[kwd][ind]=intnumbers[0]
      except:
        continue
      #no levels found in text, search in "centro"
    if len(lvl)==0:
      kwd = search_in_centro(ind)
      lvl.append(kwd)
      data[kwd[0]][ind]=intnumbers[0]
print('Datos correspondientes al',pubdate.text)    
data

Datos correspondientes al viernes, 18 de septiembre de 2020


,Centro,Incidencia,Infantil,Primaria,Secundaria,Bachillerato,CEE,FP
0,CEIP Gil Tarín (La Muela),1 aula de Infantil,1,,,,,
1,CEIP Ciudad de Zaragoza (Zaragoza),1 aula de Infantil,1,,,,,
2,CEIP Juan XXIII (Zaragoza),1 aula de Primaria,,1,,,,
3,CEIP Lucien Briet (Zaragoza),1 aula de Primaria,,1,,,,
4,Colegio Santo Domingo de Silos (Zaragoza),2 aulas de Infantil,2,,,,,
5,Colegio María Auxiliadora-Salesianas (Zaragoza),1 aula de Infantil,1,,,,,
6,Colegio El Salvador (Zaragoza),2 aulas de Primaria,,2,,,,
7,Colegio Escolapias Calasanz (Zaragoza),1 aula de Primaria,,1,,,,
8,CRA Los Bañales (Sádaba),1 aula de Primaria,,1,,,,
9,CEIP Mateo Valero (Alfamén),1 aula de Primaria,,1,,,,


In [244]:
data[['Infantil', 'Primaria', 'Secundaria',
       'Bachillerato', 'CEE', 'FP']] = data[['Infantil', 'Primaria', 'Secundaria',
       'Bachillerato', 'CEE','FP']].apply(pd.to_numeric).fillna(0)#.sum(axis=0)

In [245]:
data.sum()

Centro          CEIP Gil Tarín (La Muela)CEIP Ciudad de Zarago...
Incidencia       1 aula de Infantil 1 aula de Infantil 1 aula ...
Infantil                                                       10
Primaria                                                        9
Secundaria                                                      0
Bachillerato                                                    0
CEE                                                             0
FP                                                              1
dtype: object